In [58]:
"""
DISCLAIMER: This project is heavily inspired by DataDrivenInvestor's medium article on scraping from betting websites to find
surebets. Find this article at: https://medium.datadriveninvestor.com/make-money-with-python-the-sports-arbitrage-project-3b09d81a0098
"""
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# changing chromedriver default options
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080')  # Headless = True

web = 'https://www.sportybet.com/ng/sport/football/today'
path = '/Users/jcool/chromedriver_win32/chromedriver.exe'  # introduce your file's path inside '...'

# execute chromedriver with edited options
driver = webdriver.Chrome(path, options=options)
driver.get(web)

teams = []
x12 = []  # wins,draws, losses
btts = []
over_under = []
odds_events = []
market_tabs = []  # dropdowns
market_tabs2 = []  # dropdowns
matches_dicts = []

"""TODO: Add functionality for switching pages when there are more games"""
try:
    pagination_switch_container = WebDriverWait(driver, 4).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'pagination')))

except Exception as e:
    print(e)
    pagination_switch_container = []  # Set pagination
    print('Only one page exists')

for entry in driver.get_log('browser'):
    print(entry)

league_wraps = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-league')))

for league_wrap in league_wraps:
    # append all the market items (3way,double chance) to market_tabs array and click on it
    market_tabs.append(league_wrap.find_elements_by_class_name("market-item"))
    market_tabs[league_wraps.index(league_wrap)][2].click()
    """for match in league_wrap:
        home_team = league_wrap[match].find_elements_by_class_name('home-team')
        away_team = league_wrap[match].find_elements_by_class_name('away-team')
        teams.append([home_team,away_team])"""

match_rows = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-row')))
count = 0
for match_row in match_rows:
    count += 1
    home_team = match_row.find_element_by_class_name('home-team').text
    away_team = match_row.find_element_by_class_name('away-team').text
    parent = match_row.find_element_by_xpath('./..')  # immediate parent node
    # update 4 (+3 times .find_element_by_xpath('./..'))
    grandparent = parent.find_element_by_xpath('./..')
    league_container = grandparent.find_element_by_class_name('league-title')
    league = league_container.find_element_by_class_name('text').text

    odds = match_row.find_elements_by_class_name('m-outcome-odds')
    try:
        home_odds = float(odds[0].text)  # home comes first
        away_odds = float(odds[1].text)  # away comes second
    except Exception as e:
        print('error was at', match_rows.index(match_row), e)

    matches_dicts.append(
        {'id': match_rows.index(match_row) + 1, 'Match': f'{home_team} vs {away_team}', 'game_type': 'gg/ng',
         'home_odds': home_odds, 'away_odds': away_odds, 'league': league})
print("scraped first page")
if len(pagination_switch_container) >= 1:  # pagination renders conditionally. Execute the below logic only if its found
    for pagination in pagination_switch_container:
        pagination_switch = pagination.find_elements_by_class_name("pageNum")
    print("pagination count:", len(pagination_switch))
    if len(
            pagination_switch) > 3:  # if only one page, 3 things have 'pageNum' class name, previous icon, current page (1) and next icon
        print('switching page...')
        pagination_switch[2].click()  # click on the number 2 button
        time.sleep(2)

        league_wraps2 = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-league')))

        for league_wrap in league_wraps2:
            # append all the market items (3way,double chance) to market_tabs array and click on it
            market_tabs2.append(league_wrap.find_elements_by_class_name("market-item"))
            market_tabs2[league_wraps2.index(league_wrap)][2].click()
            """for match in league_wrap:
                home_team = league_wrap[match].find_elements_by_class_name('home-team')
                away_team = league_wrap[match].find_elements_by_class_name('away-team')
                teams.append([home_team,away_team])"""

        match_rows2 = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-row')))

        for match_row in match_rows2:
            home_team = match_row.find_element_by_class_name('home-team').text
            away_team = match_row.find_element_by_class_name('away-team').text
            odds = match_row.find_elements_by_class_name('m-outcome-odds')
            parent = match_row.find_element_by_xpath('./..')  # immediate parent node
            # update 4 (+3 times .find_element_by_xpath('./..'))
            grandparent = parent.find_element_by_xpath('./..')
            league_container = grandparent.find_element_by_class_name('league-title')
            league = league_container.find_element_by_class_name('text').text
            try:
                home_odds = float(odds[0].text)  # home comes first
                away_odds = float(odds[1].text)  # away comes second

            except Exception as e:
                print('error was at', match_rows2.index(match_row) + len(match_rows), e)
            matches_dicts.append(
                {'id': match_rows2.index(match_row) + len(match_rows) + 1, 'Match': f'{home_team} vs {away_team}',
                 'game_type': 'gg/ng', 'home_odds': home_odds, 'away_odds': away_odds, 'league': league})

    else:
        pass

# next step: get home and away odds as well as team names in a dict and append it into a new array
'''for i in matches_dicts:
    print(i)'''
'''If a league has SRL add a whole new extra string to make matching better'''
u19leagues = [
    "International Youth UEFA Youth League"
]

women_leagues = [
    "Spain Primera Division Women", "Finland Kansallinen Liiga, Women", "Scotland Premier League 1, Women",
    "England Amateur Fa Cup, Women"
]

simulated_leagues = ["Simulated Reality League UEFA Champions League SRL"]

# Add more context to Sporty team names to make string matching more accurate
for item in matches_dicts:
    if item["league"] in u19leagues:
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " U19"
        item["Match"] = " vs ".join(new_match)

    elif item["league"] in simulated_leagues:
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Simulated League)"
        item["Match"] = " vs ".join(new_match)
        
    elif item["league"] in women_leagues:
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Women)"
        item["Match"] = " vs ".join(new_match)
        
    else: 
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Main)"
        item["Match"] = " vs ".join(new_match)

sporty_df = pd.DataFrame(matches_dicts)
# sporty_df = sporty_df[~sporty_df["Match"].str.contains('SRL')]
print('done')



















{'level': 'WARNING', 'message': 'https://www.sportybet.com/ng/sport/football/today 108:20 "Registration failed with TypeError: Cannot read properties of null (reading \'postMessage\')"', 'source': 'console-api', 'timestamp': 1632927902128}
error was at 50 could not convert string to float: ''
error was at 64 list index out of range
error was at 65 list index out of range
error was at 66 list index out of range
scraped first page
pagination count: 4
switching page...
error was at 100 list index out of range
error was at 105 list index out of range
error was at 107 list index out of range
error was at 108 could not convert string to float: ''
error was at 119 list index out of range
error was at 123 list index out of range
error was at 124 list index out of range
done


In [62]:
sporty_df[sporty_df["Match"].str.contains('Juventus')]

,id,Match,game_type,home_odds,away_odds,league
4,5,Juventus (Main) vs Chelsea (Main),gg/ng,1.98,1.80,International Clubs UEFA Champions League
128,129,Juventus FC (Main) vs AS Giana Erminio (Main),gg/ng,1.89,1.75,"Italy Serie C, Group A"


In [61]:
"""
DISCLAIMER: This project is heavily inspired by DataDrivenInvestor's medium article on scraping from betting websites to find
surebets. Find this article at: https://medium.datadriveninvestor.com/make-money-with-python-the-sports-arbitrage-project-3b09d81a0098
"""
from re import search
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

# changing chromedriver default options
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080')  # Headless = True

web = 'https://sports.bet9ja.com/sport/soccer/1'
path = '/Users/jcool/chromedriver_win32/chromedriver.exe'  # introduce your file's path inside '...'

# execute chromedriver with edited options
driver = webdriver.Chrome(path, options=options)
driver.get(web)

teams = []
x12 = []  # wins,draws, losses
btts = []
over_under = []
odds_events = []
market_tabs = []  # dropdowns
matches_dicts = []

for entry in driver.get_log('browser'):
    print(entry)
# the top level bar that contains Highlights and Upcoming
matches_section = WebDriverWait(driver, 5).until \
    (EC.presence_of_all_elements_located((By.CLASS_NAME, 'sports-view__nav')))
for item in matches_section:
    matches_group_tabs = item.find_elements_by_class_name('sports-view__nav-tab-item')
    matches_group_tabs[1].click()
    print(f'Switched {matches_group_tabs[1].text} tab successfully')
    print("Next: Click on GG/NG tab")

markets_section = WebDriverWait(driver, 5).until \
    (EC.presence_of_all_elements_located(
        (By.XPATH, '//*[@id="wrapper"]/main/div/div/div/div[2]/div/div/div[1]/div[3]')))
height = driver.execute_script("return document.body.scrollHeight")
for item in markets_section:
    btts_tab = item.find_element_by_xpath("//td[@title='GG/NG']")
    btts_tab.click()
    print(f'Clicked on {btts_tab.text} tab successfully')

for i in range(height):
    driver.execute_script(f'window.scrollBy(0,20)')
    height = driver.execute_script("return document.body.scrollHeight")

matches_container = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sports-table')))
for item in matches_container:
    match_rows = item.find_elements_by_class_name('table-f')

print("scrapping odds...")
height = driver.execute_script("return document.body.scrollHeight")

for match_row in match_rows:

    try:

        home_team = match_row.find_element_by_class_name('sports-table__home').text
        away_team = match_row.find_element_by_class_name('sports-table__away').text
        odds = match_row.find_elements_by_class_name('sports-table__odds-item')
        # accomodate possibility of empty odds...
        if odds[0].text == '':
            home_odds = 0.0
            away_odds = 0.0
        else:
            # below are ternary operators to catch empty odds
            home_odds = float(odds[0].text) if type(float(odds[0].text)) == float else 'N/A'
            away_odds = float(odds[1].text) if type(float(odds[1].text)) == float else 'N/A'
        matches_dicts.append({'id': match_rows.index(match_row) + 1, 'Match': f'{home_team} vs {away_team}', 'game_type': 'gg/ng',
              'home_odds': home_odds, 'away_odds': away_odds})

    except Exception as e:
        print(f'Error was at item {match_rows.index(match_row) + 1}: {e}')  # index + 1 = Id

print("Done! Successful")

for item in matches_dicts:

    if bool(search("Srl", item["Match"])):
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Simulated League)"

        item["Match"] = " vs ".join(new_match)
        
    elif bool(search("Women", item["Match"])):
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Women)"

        item["Match"] = " vs ".join(new_match)
        
    elif bool(search("U19", item["Match"])):
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += ""
        item["Match"] = " vs ".join(new_match)
        
    else: 
        new_match = item["Match"].split(" vs ")
        for team in new_match:
            new_match[new_match.index(team)] += " (Main)"

        item["Match"] = " vs ".join(new_match)




bet9ja_df = pd.DataFrame(matches_dicts)
#bet9ja_df = bet9ja_df[bet9ja_df["Match"].str.contains('Srl')]
































{'level': 'WARNING', 'message': 'https://static.hotjar.com/c/hotjar-95609.js?sv=7 2:1244 "Hotjar not launching due to suspicious userAgent:" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/93.0.4577.82 Safari/537.36"', 'source': 'console-api', 'timestamp': 1632927973382}
Switched UPCOMING tab successfully
Next: Click on GG/NG tab
Clicked on GG/NG tab successfully
scrapping odds...
Done! Successful


In [44]:

bet9ja_df = bet9ja_df[~bet9ja_df["Match"].str.contains('Srl')]
sporty_df = sporty_df[~sporty_df["Match"].str.contains('SRL')]

In [63]:
import pickle
from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve

sporty_teams = sporty_df['Match'].tolist()
b9ja_teams = bet9ja_df['Match'].tolist()
sporty_df[['Bet9ja_Matches_Matched', 'Score_b9ja']] = sporty_df['Match'].apply(lambda x:process.extractOne(x, b9ja_teams, scorer=fuzz.token_set_ratio)).apply(pd.Series)
surebet_sporty_bet9ja = pd.merge(sporty_df, bet9ja_df, left_on='Bet9ja_Matches_Matched', right_on='Match')
surebet_sporty_bet9ja = surebet_sporty_bet9ja[surebet_sporty_bet9ja['Score_b9ja']>90]
surebet_sporty_bet9ja = surebet_sporty_bet9ja[['Match_x', 'home_odds_x','away_odds_x', 'Match_y', 'home_odds_y', 'away_odds_y']]
def find_surebet(frame):
    frame['home_odds_x'] = frame['home_odds_x'].apply(pd.Series).astype(float)
    frame['away_odds_x'] = frame['away_odds_x'].apply(pd.Series).astype(float)
    frame['home_odds_y'] = frame['home_odds_y'].apply(pd.Series).astype(float)
    frame['away_odds_y'] = frame['away_odds_y'].apply(pd.Series).astype(float)
    
   
    frame['sure_btts1'] = (1 / frame['home_odds_x']) + (1 / frame['away_odds_y'])
    frame['sure_btts2'] = (1 / frame['away_odds_x']) + (1 / frame['home_odds_y'])
    frame = frame[['Match_x', 'home_odds_x','away_odds_x' , 'Match_y','home_odds_y' ,'away_odds_y', 'sure_btts1','sure_btts2']]
    frame = frame[(frame['sure_btts1'] < 1)| (frame['sure_btts2'] < 1)]
    frame.reset_index(drop=True, inplace=True)
    return frame
surebet_sporty_bet9ja = find_surebet(surebet_sporty_bet9ja)
dict_surebet = {'sporty-bet9ja': surebet_sporty_bet9ja}
def beat_bookies(odds1, odds2, total_stake):
    x, y = symbols('x y')
    eq1 = Eq(x + y - total_stake, 0) # total_stake = x + y
    eq2 = Eq((odds2*y) - odds1*x, 0) # odds1*x = odds2*y
    stakes = solve((eq1,eq2), (x, y))
    total_investment = stakes[x] + stakes[y]
    profit1 = odds1*stakes[x] - total_stake
    profit2 = odds2*stakes[y] - total_stake
    benefit1 = f'{profit1 / total_investment * 100:.2f}%'
    benefit2 = f'{profit2 / total_investment * 100:.2f}%'
    dict_gambling = {'GG_Odds1':odds1, 'NG_Odds2':odds2, 'SportyBetStake':f'${stakes[x]:.0f}', 'Bet9jaStake':f'${stakes[y]:.0f}', 'Profit1':f'${profit1:.2f}', 'Profit2':f'${profit2:.2f}',
                    'Benefit1': benefit1, 'Benefit2': benefit2}
    return dict_gambling

total_stake = 100 #set your total stake
for frame in dict_surebet:
    if len(dict_surebet[frame])>=1:
        print('------------------SUREBETS Found! '+ frame +' (check team names)--------------------------------------------------')
        print(dict_surebet[frame])
        print('------------------Stakes-------------------------')
        for i, value in enumerate(dict_surebet[frame]['sure_btts1']):
            if value<1:
                odds1 = float(dict_surebet[frame].at[i, 'home_odds_x'])
                odds2 = float(dict_surebet[frame].at[i, 'away_odds_y'])
                teams = dict_surebet[frame].at[i, 'Match_x'].split(' vs ')
                dict_1 = beat_bookies(odds1, odds2, total_stake)
                print(str(i)+' '+'-'.join(teams)+ '(sporty->bet9ja) ----> '+ ' '.join('{}:{}'.format(x, y) for x,y in dict_1.items()))
        for i, value in enumerate(dict_surebet[frame]['sure_btts2']):
            if value<1:
                odds1 = float(dict_surebet[frame].at[i, 'away_odds_x'])
                odds2 = float(dict_surebet[frame].at[i, 'home_odds_y'])
                teams = dict_surebet[frame].at[i, 'Match_y'].split(' vs ')
                dict_1 = beat_bookies(odds1, odds2, total_stake)
                print(str(i)+' '+'-'.join(teams)+ '(bet9ja->sporty) ----> '+ ' '.join('{}:{}'.format(x, y) for x,y in dict_1.items()))


------------------SUREBETS Found! sporty-bet9ja (check team names)--------------------------------------------------
                                             Match_x  home_odds_x  \
0        FC Fiorentino (Main) vs SP Tre Fiori (Main)         1.95   
1  SS Folgore Falciano Calcio (Main) vs AC Libert...         1.95   
2            Tampereen Ilves (Women) vs Jypk (Women)         1.52   
3      Sao Paulo Fc Sp (Main) vs Santos FC SP (Main)         1.97   

   away_odds_x                                            Match_y  \
0         1.79              Fiorentino (Main) vs Tre Fiori (Main)   
1         1.79  Folgore Falciano Calcio (Main) vs Libertas (Main)   
2         2.20              Tampereen Ilves (Main) vs Jypk (Main)   
3         1.69      SAO Paulo FC SP (Main) vs Santos FC SP (Main)   

   home_odds_y  away_odds_y  sure_btts1  sure_btts2  
0         1.54         2.24    0.959249    1.208010  
1         1.61         2.11    0.986754    1.179777  
2         1.87         1.75  

In [28]:
dict_surebet

{'sporty-bet9ja':                                      Match_x  home_odds_x  away_odds_x  \
 0            Bayern Munich vs FC Dynamo Kiev         1.55         2.25   
 1                        Juventus vs Chelsea         2.05         1.76   
 2                       Wolfsburg vs Sevilla         1.81         1.97   
 3               Juventus Turin vs Chelsea FC         1.47         2.45   
 4                VfL Wolfsburg vs Sevilla FC         1.47         2.45   
 5              FC Fiorentino vs SP Tre Fiori         2.10         1.58   
 6  SS Folgore Falciano Calcio vs AC Libertas         2.10         1.58   
 7                     SS Virtus vs SS Murata         2.10         1.58   
 8            IFK Stocksund vs IFK Lidingo FK         1.65         2.00   
 
                                Match_y  home_odds_y  away_odds_y  sure_btts1  \
 0         Bayern Munich vs Dynamo Kiev         2.10         1.73    1.223196   
 1          Juventus U19 vs Chelsea U19         1.44         2.52    

In [38]:
surebet_sporty_bet9ja

,Match_x,home_odds_x,away_odds_x,Match_y,home_odds_y,away_odds_y,sure_btts1,sure_btts2
0,Man Utd vs Villarreal,1.91,1.86,Manchester Utd U19 vs Villarreal U19,1.57,2.20,0.978106,1.174577
1,Salzburg vs Lille,1.79,1.99,FC Salzburg U19 vs Lille U19,1.26,3.35,0.857167,1.296163
2,FC Fiorentino vs SP Tre Fiori,2.10,1.58,Fiorentino vs Tre Fiori,1.54,2.24,0.922619,1.282262
3,SS Folgore Falciano Calcio vs AC Libertas,2.10,1.58,Folgore Falciano Calcio vs Libertas,1.61,2.11,0.950124,1.254029
4,SS Virtus vs SS Murata,2.10,1.58,SS Virtus vs Murata,1.70,1.98,0.981241,1.221147


In [33]:
"""
DISCLAIMER: This project is heavily inspired by DataDrivenInvestor's medium article on scraping from betting websites to find
surebets. Find this article at: https://medium.datadriveninvestor.com/make-money-with-python-the-sports-arbitrage-project-3b09d81a0098
"""
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

#changing chromedriver default options
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080') #Headless = True

web = 'https://www.sportybet.com/ng/sport/football/today'
path = '/Users/jcool/chromedriver_win32/chromedriver.exe' #introduce your file's path inside '...'

#execute chromedriver with edited options
driver = webdriver.Chrome(path, options=options)
driver.get(web)

teams = []
x12 = []  # wins,draws, losses
btts = []
over_under=[]
odds_events = []
market_tabs = [] # dropdowns
market_tabs2 = [] # dropdowns
matches_dicts = []

"""TODO: Add functionality for switching pages when there are more games"""
try:
    pagination_switch_container = WebDriverWait(driver, 4).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'pagination')))

except Exception as e:
    print(e)
    pagination_switch_container = []  # Set pagination
    print('Only one page exists')


for entry in driver.get_log('browser'):
    print(entry)

league_wraps = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-league')))

for league_wrap in league_wraps:

    # append all the market items (3way,double chance) to market_tabs array and click on it
    market_tabs.append(league_wrap.find_elements_by_class_name("market-item"))
    market_tabs[league_wraps.index(league_wrap)][2].click()
    """for match in league_wrap:
        home_team = league_wrap[match].find_elements_by_class_name('home-team')
        away_team = league_wrap[match].find_elements_by_class_name('away-team')
        teams.append([home_team,away_team])"""

match_rows = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-row')))
count = 0
for match_row in match_rows:
    count+=1
    home_team = match_row.find_element_by_class_name('home-team').text
    away_team = match_row.find_element_by_class_name('away-team').text
    parent = match_row.find_element_by_xpath('./..')  # immediate parent node
    # update 4 (+3 times .find_element_by_xpath('./..'))
    grandparent = parent.find_element_by_xpath('./..')
    league_container = grandparent.find_element_by_class_name('league-title')
    league = league_container.find_element_by_class_name('text').text

    odds = match_row.find_elements_by_class_name('m-outcome-odds')
    try:
        home_odds = float(odds[0].text) # home comes first
        away_odds = float(odds[1].text) # away comes second
    except Exception as e:
        print('error was at', match_rows.index(match_row), e)


    matches_dicts.append({'id': match_rows.index(match_row)+1,'Match':f'{home_team} vs {away_team}', 'game_type': 'gg/ng', 'home_odds': home_odds, 'away_odds': away_odds, 'league': league})
print("scraped first page")
if len(pagination_switch_container) >= 1: # pagination renders conditionally. Execute the below logic only if its found
    for pagination in pagination_switch_container:
        pagination_switch = pagination.find_elements_by_class_name("pageNum")
    print("pagination count:", len(pagination_switch))
    if len(pagination_switch) > 3: # if only one page, 3 things have 'pageNum' class name, previous icon, current page (1) and next icon
        print('switching page...')
        pagination_switch[2].click() # click on the number 2 button
        time.sleep(2)

        league_wraps2 = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-league')))

        for league_wrap in league_wraps2:
            # append all the market items (3way,double chance) to market_tabs array and click on it
            market_tabs2.append(league_wrap.find_elements_by_class_name("market-item"))
            market_tabs2[league_wraps2.index(league_wrap)][2].click()
            """for match in league_wrap:
                home_team = league_wrap[match].find_elements_by_class_name('home-team')
                away_team = league_wrap[match].find_elements_by_class_name('away-team')
                teams.append([home_team,away_team])"""

        match_rows2 = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'match-row')))

        for match_row in match_rows2:
            home_team = match_row.find_element_by_class_name('home-team').text
            away_team = match_row.find_element_by_class_name('away-team').text
            odds = match_row.find_elements_by_class_name('m-outcome-odds')
            parent = match_row.find_element_by_xpath('./..')  # immediate parent node
            # update 4 (+3 times .find_element_by_xpath('./..'))
            grandparent = parent.find_element_by_xpath('./..')
            league_container = grandparent.find_element_by_class_name('league-title')
            league = league_container.find_element_by_class_name('text').text
            try:
                home_odds = float(odds[0].text)  # home comes first
                away_odds = float(odds[1].text)  # away comes second

            except Exception as e:
                print('error was at', match_rows2.index(match_row) + len(match_rows), e)
            matches_dicts.append({'id': match_rows2.index(match_row)+len(match_rows) + 1, 'Match':f'{home_team} vs {away_team}',
                                      'game_type': 'gg/ng', 'home_odds': home_odds, 'away_odds': away_odds, 'league': league})

    else:
        pass

# next step: get home and away odds as well as team names in a dict and append it into a new array
'''for i in matches_dicts:
    print(i)'''
'''If a league has SRL add a whole new extra string to make matching better'''
u19leagues=[
    "International Youth UEFA Youth League"
]

women_leagues = [
"Spain Primera Division Women", "Finland Kansallinen Liiga, Women","Scotland Premier League 1, Women", "England Amateur Fa Cup, Women"
]

# Add more context to Sporty team names to make string matching more accurate
for item in matches_dicts:
    if item["league"] in u19leagues:
        item["Match"]+=" U19" 
    elif item["league"] in women_leagues:
        item["Match"]+=" Women"
sporty_df = pd.DataFrame(matches_dicts)
sporty_df = sporty_df[~sporty_df["Match"].str.contains('SRL')]
print('done')



















{'level': 'WARNING', 'message': 'https://www.sportybet.com/ng/sport/football/today 108:20 "Registration failed with TypeError: Cannot read properties of null (reading \'postMessage\')"', 'source': 'console-api', 'timestamp': 1632905899518}
scraped first page
pagination count: 5
switching page...
error was at 100 could not convert string to float: ''
error was at 152 list index out of range
error was at 153 list index out of range
error was at 154 list index out of range
done


In [34]:
matches_dicts

[{'id': 1,
  'Match': 'Atalanta vs Young Boys Bern',
  'game_type': 'gg/ng',
  'home_odds': 1.63,
  'away_odds': 2.25,
  'league': 'International Clubs UEFA Champions League'},
 {'id': 2,
  'Match': 'FK Zenit Saint Petersburg vs Malmo',
  'game_type': 'gg/ng',
  'home_odds': 1.81,
  'away_odds': 1.97,
  'league': 'International Clubs UEFA Champions League'},
 {'id': 3,
  'Match': 'Bayern Munich vs FC Dynamo Kiev',
  'game_type': 'gg/ng',
  'home_odds': 2.2,
  'away_odds': 1.65,
  'league': 'International Clubs UEFA Champions League'},
 {'id': 4,
  'Match': 'Benfica vs Barcelona',
  'game_type': 'gg/ng',
  'home_odds': 1.74,
  'away_odds': 2.05,
  'league': 'International Clubs UEFA Champions League'},
 {'id': 5,
  'Match': 'Juventus vs Chelsea',
  'game_type': 'gg/ng',
  'home_odds': 2.0,
  'away_odds': 1.77,
  'league': 'International Clubs UEFA Champions League'},
 {'id': 6,
  'Match': 'Man Utd vs Villarreal',
  'game_type': 'gg/ng',
  'home_odds': 1.91,
  'away_odds': 1.86,
  'leagu